## Custom Environment - Guidance


This notebook provides you instructions how to resolve issues, in case you are running this outside of a workshop on your own AWS environment.

To run this notebook and all notebooks in the workshop please use the `Python 3` kernel in JupyterLab

## Setup
Get the latest version of SageMaker Python SDK.

<div class="alert alert-info"> 💡 The workshop and all notebooks were tested with Sagemaker Distribution `1.11` and the SageMaker Python SDK (the package sagemaker) version 2.219.0. The notebooks don't pin the version of the sagemaker. If you encounter any incompatibility issues, you can install the specific version of the sagemaker by running the pip command: <code>%pip install sagemaker=2.219.0</code>
</div>

### Import packages

In [1]:
import time
import os
import json
import boto3
import numpy as np  
import pandas as pd 
import sagemaker
from time import gmtime, strftime, sleep

(sagemaker.__version__,boto3.__version__)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


('2.232.2', '1.35.49')

### Set constants

In [2]:
# Get some variables you need to interact with SageMaker service
boto_session = boto3.Session()
region = boto_session.region_name
project_prefix = "amzn"
bucket_name = sagemaker.Session().default_bucket()
bucket_prefix = f"{bucket_name}/{project_prefix}"
sm_session = sagemaker.Session()
sm_client = boto_session.client("sagemaker")
sm_role = sagemaker.get_execution_role()

initialized = True

print(sm_role)

arn:aws:iam::762233765926:role/service-role/SageMaker-ExecutionRole-20240920T110576


In [3]:
# Store some variables to keep the value between the notebooks
%store bucket_name
%store project_prefix   
%store bucket_prefix
%store sm_role
%store region
%store initialized

Stored 'bucket_name' (str)
Stored 'project_prefix' (str)
Stored 'bucket_prefix' (str)
Stored 'sm_role' (str)
Stored 'region' (str)
Stored 'initialized' (bool)


### Get domain id
You need this value `domain_id` in many SageMaker Python SDK and boto3 SageMaker API calls. The notebook metadata file contains `domain_id` value. The following code demonstrates how to access the notebook metadata file and get the `domain_id`.

In [4]:
import sagemaker


In [5]:
NOTEBOOK_METADATA_FILE = "/opt/ml/metadata/resource-metadata.json"
domain_id = None

if os.path.exists(NOTEBOOK_METADATA_FILE):
    with open(NOTEBOOK_METADATA_FILE, "rb") as f:
        metadata = json.loads(f.read())
        domain_id = metadata.get('DomainId')
        space_name = metadata.get('SpaceName')
        print(f"SageMaker domain id: {domain_id}")

if not space_name:
    raise Exception(f"Cannot find the current space name. Make sure you run this notebook in a JupyterLab in the SageMaker Studio")
else:
    print(f"Space name: {space_name}")
    
r = sm_client.describe_space(DomainId=domain_id, SpaceName=space_name)
user_profile_name = r['OwnershipSettings']['OwnerUserProfileName']

assert(user_profile_name)
print(f"User profile: {user_profile_name}")

#todo get domain id from

%store domain_id
%store space_name
%store user_profile_name

SageMaker domain id: d-9zycgmgeyaqq
Space name: finetune-vison-llm-02
User profile: arlind-nocaj-13b
Stored 'domain_id' (str)
Stored 'space_name' (str)
Stored 'user_profile_name' (str)


### Connect to MLflow tracking server
If you're running an AWS-led workshop or used the delivered CloudFormation template to provision your workshop environment, an MLflow server must be up and running. If you don't have an MLflow server, follow the [Developer Guide](https://docs.aws.amazon.com/sagemaker/latest/dg/mlflow-create-tracking-server.html) or run the following code cell to create a new one.

To create and manage an MLflow tracking server and to work with managed MLflow experiements, you need the following permissions attached to the SageMaker execution role:

```json
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "sagemaker-mlflow:*",
                "sagemaker:CreateMlflowTrackingServer",
                "sagemaker:UpdateMlflowTrackingServer",
                "sagemaker:DeleteMlflowTrackingServer",
                "sagemaker:StartMlflowTrackingServer",
                "sagemaker:StopMlflowTrackingServer",
                "sagemaker:CreatePresignedMlflowTrackingServerUrl"
            ],
            "Resource": "*"
        }
    ]
}
```

Execute the following code to check if you have a running MLflow server.

In [10]:
def get_mlflow_server_arn():
    list_servers = boto3.client("sagemaker").list_mlflow_tracking_servers()['TrackingServerSummaries']

    assert len(list_servers)==1 # should be one MLflow server, if not make sure to set your correct one        
    mlflow_arn = list_servers[0]['TrackingServerArn']
    mlflow_name = list_servers[0]['TrackingServerName']
    print(f"MLflow Server Name: {mlflow_name},\nARN: {mlflow_arn}")
    return mlflow_name, mlflow_arn

mlflow_name, mlflow_arn = get_mlflow_server_arn()

MLflow Server Name: main-mlflow,
ARN: arn:aws:sagemaker:us-west-2:762233765926:mlflow-tracking-server/main-mlflow


<div style="border: 4px solid coral; text-align: center; margin: auto;">
Creation of an MLflow server can take up to 25 minutes. You don't need to wait - proceed with the flow of the workshop.
</div>

In [11]:
%store mlflow_arn
%store mlflow_name

Stored 'mlflow_arn' (str)
Stored 'mlflow_name' (str)


## Install Docker to enable Studio local mode
Amazon SageMaker Studio applications support the use of local mode to create estimators, processors, and pipelines, then deploy them to a local environment. With local mode, you can test machine learning scripts before running them in Amazon SageMaker managed training or hosting environments. Refer to [Local mode support in Amazon SageMaker Studio](https://docs.aws.amazon.com/sagemaker/latest/dg/studio-updated-local.html) to understand which docker operations the Studio currently supports.

To use local mode in Studio applications, you must install Docker into your JupyterLab space. 

### Check if docker access is enabled

In [12]:
# check that docker enabled in the SageMaker domain
docker_settings = sm_client.describe_domain(DomainId=domain_id)['DomainSettings'].get('DockerSettings')
docker_enabled = False

if docker_settings:
    if docker_settings.get('EnableDockerAccess') in ['ENABLED']:
        print(f"The docker access is ENABLED in the domain {domain_id}")
        docker_enabled = True

if not docker_enabled:
    raise Exception(f"You must enable docker access in the domain to use Studio local mode")

The docker access is ENABLED in the domain d-9zycgmgeyaqq


<div style="border: 4px solid coral; text-align: center; margin: auto;">
If the previous code cell raised an exeption that the docker access is not enabled, you need to enable the access. See the following instructions how to do it.
</div>

In [13]:
print(f"Domain id: {domain_id}")

Domain id: d-9zycgmgeyaqq


### Enable docker access for the SageMaker domain

<div class="alert alert-info">You only need this section if the docker access is not enabled in the domain.
</div>

You need `sagemaker:UpdateDomain` permission in the execution role. You cannot update domain from this notebook because the notebook execution role doesn't have this permission. To update domain settings, you can use one of the following options.

#### Option 1: run `update_domain` in the notebook
If you have the corresponding permissions in the notebook execution role, you can run the following code in a notebook:

```python
import boto3

r = boto3.client('sagemaker').update_domain(
    DomainId=domain_id,
    DomainSettingsForUpdate={
        'DockerSettings': {
            'EnableDockerAccess':'ENABLED',
            'VpcOnlyTrustedAccounts': []
        }
    }
)
```

#### Option 2: run `aws sagemaker` CLI in the  terminal
Make sure you run `AWS CLI` in the terminal where you have the corresponding permissions `sagemaker:UpdateDomain`. Run the following command, e.g. for us-east-1:

```
 aws sagemaker update-domain --domain-id <DOMAIN-ID> --domain-settings-for-update '{"DockerSettings": {"EnableDockerAccess": "ENABLED","VpcOnlyTrustedAccounts": []}}'
```

For example, you can run the command above in the [AWS CloudShell](https://aws.amazon.com/blogs/aws/aws-cloudshell-command-line-access-to-aws-resources/) in your AWS account.

For other regions make sure to adjust the account id as described below.

#### Determining the ECR Repo Account for for VPC only Mode

In Sagemaker domains which are in VPC-only mode, docker can only pull images from ECR repositories of your own or trusted accounts. We thus will add the default Sagemaker Distribution repository to the list of trusted accounts for your specific AWS region.

Lets first determine the public ECR repository for the sagemaker distribution for your specific AWS region.
Make sure that your role has access to get the parameter over ssm. The policy would look like this

```
{
	"Version": "2012-10-17",
	"Statement": [
		{
			"Sid": "GetECRRepoAccountID",
			"Effect": "Allow",
			"Action": [
				"ssm:GetParameter"
			],
			"Resource": [
				"arn:aws:ssm:*::parameter/aws/service/sagemaker-distribution/ecr-account-id"
			]
		}
	]
}
```

In [23]:
import boto3

sagemaker_dist_repos = "/aws/service/sagemaker-distribution/ecr-account-id"
sm_dist_repo_account = boto3.client('ssm', region_name=region).get_parameter(Name=sagemaker_dist_repos)['Parameter']['Value']

SM_DIST_IMAGE=f"{sm_dist_repo_account}.dkr.ecr.{region}.amazonaws.com/sagemaker-distribution-prod:1.11.0-gpu"
print(f"Region: {REGION}, Sagemaker distribution account: {sm_dist_repo_account}")
print(f"SM_DIST_IMAGE: {SM_DIST_IMAGE}")
print(f"""'VpcOnlyTrustedAccounts': ["{sm_dist_repo_account}"] """)

Region: us-east-1, Sagemaker distribution account: 542918446943
SM_DIST_IMAGE: 542918446943.dkr.ecr.us-west-2.amazonaws.com/sagemaker-distribution-prod:1.11.0-gpu
'VpcOnlyTrustedAccounts': ["542918446943"] 


Make sure to run Option 1 or Option 2 above with the determined SM_DIST_IMAGE.

In [24]:
# check the updated settings
sm_client.describe_domain(DomainId=domain_id)['DomainSettings']

{'DockerSettings': {'EnableDockerAccess': 'ENABLED',
  'VpcOnlyTrustedAccounts': []},
 'AmazonQSettings': {'Status': 'DISABLED'}}

### Install Docker

Lets install the packages to utilize Docker.

In [26]:
%%bash

# see https://docs.docker.com/engine/install/ubuntu/#install-using-the-repository
sudo apt-get update
sudo apt-get install -y ca-certificates curl
sudo install -m 0755 -d /etc/apt/keyrings
sudo curl -fsSL https://download.docker.com/linux/ubuntu/gpg -o /etc/apt/keyrings/docker.asc
sudo chmod a+r /etc/apt/keyrings/docker.asc

# Add the repository to Apt sources:
echo \
  "deb [arch=$(dpkg --print-architecture) signed-by=/etc/apt/keyrings/docker.asc] https://download.docker.com/linux/ubuntu \
  $(. /etc/os-release && echo "$VERSION_CODENAME") stable" | \
  sudo tee /etc/apt/sources.list.d/docker.list > /dev/null
sudo apt-get update

## Currently only Docker version 20.10.X is supported in Studio: see https://docs.aws.amazon.com/sagemaker/latest/dg/studio-updated-local.html
# pick the latest patch from:
# apt-cache madison docker-ce | awk '{ print $3 }' | grep -i 20.10
VERSION_STRING=5:20.10.24~3-0~ubuntu-jammy
sudo apt-get install docker-ce-cli=$VERSION_STRING docker-compose-plugin -y

# validate the Docker Client is able to access Docker Server at [unix:///docker/proxy.sock]
docker version

Hit:1 https://download.docker.com/linux/ubuntu jammy InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Reading package lists...
Reading package lists...
Building dependency tree...
Reading state information...
ca-certificates is already the newest version (20240203~22.04.1).
curl is already the newest version (7.81.0-1ubuntu1.18).
0 upgraded, 0 newly installed, 0 to remove and 4 not upgraded.
Hit:1 https://download.docker.com/linux/ubuntu jammy InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.c

## Restart kernel

In [27]:
# Restart kernel to get the packages
import IPython
IPython.Application.instance().kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

## Further workshop flow
Continue with the workshop by going to Module 01.
